In [144]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm, linear_model
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler,PolynomialFeatures, LabelEncoder,FunctionTransformer
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from scipy import stats
%matplotlib inline
titanic = pd.read_csv('train.csv')
testdf = pd.read_csv('test.csv')

In [147]:
titanic[titanic['Cabin'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
12,13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.0500,NaN,S
13,14,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.2750,NaN,S
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.0,0,0,350406,7.8542,NaN,S


In [148]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


This is a redo of a previous titanic dataset I worked with, but this time using pipelines. 


In [2]:
titanic['Cabin'] =titanic['Cabin'].fillna(value='?')
titanic['Embarked'] =titanic['Embarked'].fillna(value='S')
titanic['Age'] =titanic['Age'].fillna(value='?')
testdf['Cabin'] =testdf['Cabin'].fillna(value='?')
testdf['Embarked'] =testdf['Embarked'].fillna(value='S')
testdf['Age'] =testdf['Age'].fillna(value='?')
testdf.loc[152,'Fare']=13.6755 # we fill in the missing fare from the test data here. 

Certain things, such as missing data, are best handled without pipelines

In [3]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,?,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,?,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,?,S


In [4]:
testdf.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,?,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7.0000,?,S
2,894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,?,Q
3,895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,?,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22,1,1,3101298,12.2875,?,S


In [48]:
def titlepull(title_string):
    title_list = title_string.split()
    for possible_title in title_list:
        if possible_title[-1] == '.':
            return possible_title
def cabinpull(cabin_string):
    if cabin_string[0]=='?':
        return '?'
    else:
        return cabin_string[0]
    
def tit_deck(df):
    df['Deck']=df.Cabin.apply(cabinpull)
    deck_dummies = pd.get_dummies(df.Deck,prefix='Deck')
    return deck_dummies

tit_deck_tf = FunctionTransformer(tit_deck,validate=False)

def tit_class(df):
    pclass_dummies=   pd.get_dummies(df.Pclass,prefix='plcass')
    return pclass_dummies

tit_class_tf = FunctionTransformer(tit_class, validate=False)

def tit_title(df):
    df['Title']= df.Name.apply(titlepull)
    title_dummies= pd.get_dummies(df.Title,prefix='Title')
    return title_dummies

tit_title_tf  = FunctionTransformer(tit_title, validate=False)


def tit_features(df):
    df['IsFemale'] = (df.Sex=='female').astype(int)
    return df[['IsFemale','Fare','Pclass']]

tit_features_tf  = FunctionTransformer(tit_features, validate=False)

def tit_family(df):
    df['FamilyCount']= (df['SibSp'])+(df['Parch'])
    fam_dummies=  pd.get_dummies(df.FamilyCount,prefix='family')
    return fam_dummies
               
tit_family_tf  = FunctionTransformer(tit_family, validate=False)

In [49]:

age_transformers = [('family',tit_family_tf),('class',tit_class_tf),('deck',tit_deck_tf),('features',tit_features_tf),('title',tit_title_tf)]

In [50]:
fu_age_impute = FeatureUnion(age_transformers)

The first thing we want to do is make a pipeline for linear regression or KNN regression that predicts ages. 


In [51]:
fu_age_impute.fit_transform(titanic)

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [53]:
fu_age_impute.fit_transform(testdf)

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  1.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])

In [10]:
knr = KNeighborsRegressor(n_jobs=-1)
lr = LinearRegression()
ss = StandardScaler()

In [11]:
age_impute_pipe_kn =  Pipeline(
    [
        ('fu_age',fu_age_impute),
        ('ss',ss)
        ('knr',knr)
    ]

)

In [12]:
age_impute_pipe_lr =  Pipeline(
    [
        ('fu',fu_age_impute),
        ('ss',ss),
        ('lr',lr)
    ]

)

In [13]:
titanic_ages =titanic[(titanic['Age']!='?')]
titanic_no_ages = titanic[(titanic['Age']=='?')]

test_ages =testdf[(testdf['Age']!='?')]
test_no_ages = testdf[(testdf['Age']=='?')]


In [14]:
age_params = {
    
}

In [54]:
titanic_ages.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'FamilyCount',
       'IsFemale', 'Deck', 'Title'],
      dtype='object')

In [56]:
fu_age_impute.fit_transform(test_no_ages)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

array([[ 1.,  0.,  0., ...,  1.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  1., ...,  1.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  1.,  0.,  0.],
       [ 1.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])

In [57]:
age_impute_X = titanic_ages[['Pclass', 'Name', 'Sex', 'SibSp','Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']]
age_impute_y = titanic_ages['Age']
test_impute_X = titanic_no_ages[['Pclass', 'Name', 'Sex', 'SibSp','Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']]
test_impute_y = titanic_no_ages['Age']


In [17]:
gs_age_kn = GridSearchCV(age_impute_pipe_kn, param_grid=age_params, cv=5)
gs_age_lr = GridSearchCV(age_impute_pipe_lr, param_grid=age_params, cv=5)

In [58]:
age_impute_pipe_kn.fit(age_impute_X,age_impute_y)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

Pipeline(steps=[('fu_age', FeatureUnion(n_jobs=1,
       transformer_list=[('age_features_tf', FunctionTransformer(accept_sparse=False,
          func=<function age_features at 0x7f276eff28c8>, inv_kw_args=None,
          inverse_func=None, kw_args=None, pass_y=False, validate=False))],
       transformer_weights=None)), ('knr', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
          weights='uniform'))])

In [59]:
age_impute_pipe_kn.score(age_impute_X,age_imput_y)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

0.39831960968725022

In [60]:
age_impute_pipe_kn.predict(test_impute_X)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/opt/conda/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/opt/conda/lib/python3.6/runpy.py in _run

So here is what has happened causing this failure: Because I use the pandas function get dummies, there are missing dummy values in between the training and test set (a different number of titles, etc). There are a few options.  The easiest is probably to switch from dummies to label encoding, the other is to add dummies in the missing data stages(treating a missing label as that)

In [61]:
gs_age_kn.fit(age_impute_X,age_imput_y)
print(gs.best_score_)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/opt/conda/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/opt/conda/lib/python3.6/runpy.py in _run

Grid search also fails here because at every cross validation step there are different dummies files. 
Let's redo this with dummies first. 

In [104]:
titanic2 = pd.read_csv('train.csv')
testdf2 = pd.read_csv('test.csv')

In [105]:
titanic2['Cabin'] =titanic2['Cabin'].fillna(value='?')
titanic2['Embarked'] =titanic2['Embarked'].fillna(value='S')
titanic2['Age'] =titanic2['Age'].fillna(value='?')
testdf2['Cabin'] =testdf2['Cabin'].fillna(value='?')
testdf2['Embarked'] =testdf2['Embarked'].fillna(value='S')
testdf2['Age'] =testdf2['Age'].fillna(value='?')
testdf2.loc[152,'Fare']=13.6755 # we fill in the missing fare from the test data here. 

In [106]:
titanic2['Deck']=titanic2.Cabin.apply(cabinpull)
titanic2['Title']= titanic2.Name.apply(titlepull)
testdf2['Deck']=testdf2.Cabin.apply(cabinpull)
testdf2['Title']= testdf2.Name.apply(titlepull)
titanic2['FamilyCount']= (titanic2['SibSp'])+(titanic2['Parch'])
testdf2['FamilyCount']= (testdf2['SibSp'])+(testdf2['Parch'])

In [107]:
embarked_dummies= pd.get_dummies(titanic2.Embarked, prefix='Embarked_')
title_dummies= pd.get_dummies(titanic2.Title,prefix='Title')
parc_dummies=  pd.get_dummies(titanic2.Parch,prefix='parch')
pclass_dummies=   pd.get_dummies(titanic2.Pclass,prefix='plcass')
sibsp_dummies=  pd.get_dummies(titanic2.SibSp,prefix='sibsp')
family_dummies = pd.get_dummies(titanic2.FamilyCount,prefix='family')
deck_dummies = pd.get_dummies(titanic2.Deck,prefix='Deck')

test_embarked_dummies= pd.get_dummies(testdf2.Embarked, prefix='Embarked_')
test_title_dummies= pd.get_dummies(testdf2.Title,prefix='Title')
test_parc_dummies=  pd.get_dummies(testdf2.Parch,prefix='parch')
test_pclass_dummies=   pd.get_dummies(testdf2.Pclass,prefix='plcass')
test_sibsp_dummies=  pd.get_dummies(testdf2.SibSp,prefix='sibsp')
testfamily_dummies = pd.get_dummies(testdf2.FamilyCount,prefix='family')
testdeck_dummies = pd.get_dummies(testdf2.Deck,prefix='Deck')

In [108]:
dummieslist = [title_dummies,embarked_dummies,titanic2, parc_dummies,pclass_dummies,sibsp_dummies,family_dummies,deck_dummies]
titanic_featured= pd.concat(dummieslist, axis=1)
testdummieslist = [test_title_dummies,test_embarked_dummies,testdf2, test_parc_dummies,test_pclass_dummies,test_sibsp_dummies,testfamily_dummies,testdeck_dummies]
test_featured= pd.concat(testdummieslist, axis=1)

In [109]:
titanic_featured.shape

(891, 70)

ValueError: fill value must be in categories

In [110]:
test_featured.shape

(418, 61)

So the problem we have is the missing dummies

In [74]:
titanic_featured.columns

Index(['Title_Capt.', 'Title_Col.', 'Title_Countess.', 'Title_Don.',
       'Title_Dr.', 'Title_Jonkheer.', 'Title_Lady.', 'Title_Major.',
       'Title_Master.', 'Title_Miss.', 'Title_Mlle.', 'Title_Mme.',
       'Title_Mr.', 'Title_Mrs.', 'Title_Ms.', 'Title_Rev.', 'Title_Sir.',
       'Embarked__C', 'Embarked__Q', 'Embarked__S', 'PassengerId', 'Survived',
       'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked', 'Deck', 'Title', 'FamilyCount', 'parch_0',
       'parch_1', 'parch_2', 'parch_3', 'parch_4', 'parch_5', 'parch_6',
       'plcass_1', 'plcass_2', 'plcass_3', 'sibsp_0', 'sibsp_1', 'sibsp_2',
       'sibsp_3', 'sibsp_4', 'sibsp_5', 'sibsp_8', 'family_0', 'family_1',
       'family_2', 'family_3', 'family_4', 'family_5', 'family_6', 'family_7',
       'family_10', 'Deck_?', 'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E',
       'Deck_F', 'Deck_G', 'Deck_T'],
      dtype='object')

In [75]:
test_featured.columns

Index(['Title_Col.', 'Title_Dona.', 'Title_Dr.', 'Title_Master.',
       'Title_Miss.', 'Title_Mr.', 'Title_Mrs.', 'Title_Ms.', 'Title_Rev.',
       'Embarked__C', 'Embarked__Q', 'Embarked__S', 'PassengerId', 'Pclass',
       'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin',
       'Embarked', 'Deck', 'Title', 'FamilyCount', 'parch_0', 'parch_1',
       'parch_2', 'parch_3', 'parch_4', 'parch_5', 'parch_6', 'parch_9',
       'plcass_1', 'plcass_2', 'plcass_3', 'sibsp_0', 'sibsp_1', 'sibsp_2',
       'sibsp_3', 'sibsp_4', 'sibsp_5', 'sibsp_8', 'family_0', 'family_1',
       'family_2', 'family_3', 'family_4', 'family_5', 'family_6', 'family_7',
       'family_10', 'Deck_?', 'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E',
       'Deck_F', 'Deck_G'],
      dtype='object')

In [123]:
 new_test_featured = test_featured.reindex(columns=titanic_featured.columns, copy=False).fillna(0)

In [124]:
new_test_featured.head()

,Title_Capt.,Title_Col.,Title_Countess.,Title_Don.,Title_Dr.,Title_Jonkheer.,Title_Lady.,Title_Major.,Title_Master.,Title_Miss.,...,family_10,Deck_?,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T
0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0,...,0,1,0,0,0,0,0,0,0,0.0
1,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0,...,0,1,0,0,0,0,0,0,0,0.0
2,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0,...,0,1,0,0,0,0,0,0,0,0.0
3,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0,...,0,1,0,0,0,0,0,0,0,0.0
4,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0,...,0,1,0,0,0,0,0,0,0,0.0


In [131]:
def numeric_features(df):
    df['IsFemale'] = (df.Sex=='female').astype(int)
    return df.select_dtypes(exclude= ['object'])
numeric_features_tf = FunctionTransformer(numeric_features, validate=False)

In [136]:
age_impute_pipe_kn2 =  Pipeline(
    [
        ('num_feat',numeric_features_tf),
        ('ss',ss),
        ('knr',knr)
    ]

)

age_impute_pipe_kn2 =  Pipeline(
    [
        ('num_feat',numeric_features_tf),
        ('ss',ss),
        ('knr',knr)
    ]

)

In [138]:
titanic_ages =titanic_featured[(titanic_featured['Age']!='?')]
titanic_no_ages = titanic_featured[(titanic_featured['Age']=='?')]

test_ages =test_featured_index[(test_featured_index['Age']!='?')]
test_no_ages = test_featured_index[(test_featured_index['Age']=='?')]


In [137]:
knrange = range(1,30)
tempparams= { 'knr__n_neighbors':knrange}
gstemp  =  GridSearchCV(age_impute_pipe_kn2, param_grid=tempparams, cv=8)


In [140]:
gstemp.fit(titanic_ages.drop('Age',axis=1),titanic_ages['Age'])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

...

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

GridSearchCV(cv=8, error_score='raise',
       estimator=Pipeline(steps=[('num_feat', FunctionTransformer(accept_sparse=False,
          func=<function numeric_features at 0x7f276ee54b70>,
          inv_kw_args=None, inverse_func=None, kw_args=None, pass_y=False,
          validate=False)), ('ss', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knr', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
          weights='uniform'))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'knr__n_neighbors': range(1, 30)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [141]:
gstemp.best_score_

0.29247863022988768

In [142]:
gstemp.predict(titanic_no_ages.drop('Age',axis=1))

array([29.236842105263158, 32.473684210526315, 30.710526315789473,
       27.94736842105263, 28.763157894736842, 28.86842105263158,
       32.21052631578947, 28.763157894736842, 27.289473684210527,
       27.94736842105263, 28.86842105263158, 32.0, 28.973684210526315,
       27.210526315789473, 40.28947368421053, 36.921052631578945,
       11.416842105263159, 28.86842105263158, 28.86842105263158,
       28.973684210526315, 28.605263157894736, 28.92105263157895,
       28.710526315789473, 28.789473684210527, 25.92105263157895,
       28.710526315789473, 29.763157894736842, 19.657894736842106,
       22.236842105263158, 29.07894736842105, 29.07894736842105,
       7.631578947368421, 39.8421052631579, 34.10526315789474,
       9.342105263157896, 16.13157894736842, 28.342105263157894,
       45.026315789473685, 29.55263157894737, 29.289473684210527,
       28.81578947368421, 26.13157894736842, 32.0, 26.763157894736842,
       14.31578947368421, 25.842105263157894, 25.473684210526315,
     

In [143]:
gstemp.predict(test_no_ages.drop('Age',axis=1))

array([29.763157894736842, 41.36842105263158, 26.394736842105264,
       24.263157894736842, 24.157894736842106, 30.5, 39.473684210526315,
       28.86842105263158, 28.605263157894736, 26.36842105263158,
       31.42105263157895, 30.5, 30.5, 32.026315789473685,
       26.55263157894737, 28.86842105263158, 30.5, 30.5, 29.07894736842105,
       29.07894736842105, 30.5, 28.55263157894737, 27.81578947368421,
       31.236842105263158, 29.13157894736842, 25.763157894736842,
       30.94736842105263, 26.55263157894737, 46.421052631578945,
       35.473684210526315, 27.81578947368421, 28.55263157894737, 30.5,
       38.55263157894737, 30.5, 27.81578947368421, 29.13157894736842,
       14.526315789473685, 35.473684210526315, 30.736842105263158,
       28.55263157894737, 34.8421052631579, 30.5, 28.55263157894737, 30.5,
       20.55263157894737, 28.55263157894737, 29.13157894736842, 30.5,
       13.81157894736842, 28.55263157894737, 30.5, 29.13157894736842, 30.5,
       35.473684210526315, 30.5,

In [133]:
titanic_featured.select_dtypes(exclude= ['object']).head()

,Title_Capt.,Title_Col.,Title_Countess.,Title_Don.,Title_Dr.,Title_Jonkheer.,Title_Lady.,Title_Major.,Title_Master.,Title_Miss.,...,family_10,Deck_?,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
gs_age_lr.fit(age_impute_X,age_imput_y)
print(gs_age_lr.best_score)

In [ ]:
titanic_ages.shape

In [ ]:
age_impute_X = titanic_ages[['Pclass', 'Name', 'Sex', 'SibSp','Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']]
age_imput_y = titanic_ages['Age']

In [ ]:
age_impute_X.shape

In [ ]:
age_imput_y.shape

In [ ]:
X_train, X_text, y_train, y_test = train_test_split(titanic_ages,titanic_ages['Age'])

In [27]:
feedmetoknn = fu_age_impute.fit_transform(age_impute_X,age_imput_y)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

In [33]:
knnfeaturesscaled= ss.fit_transform(feedmetoknn)

In [ ]:
knr.fit(feedmetoknn,age_imput_y)

In [ ]:
knr.score(feedmetoknn,age_imput_y)

In [ ]:
lr.fit(feedmetoknn,age_imput_y)

In [ ]:
lr.score(feedmetoknn,age_imput_y)

In [37]:
knrange = range(1,30)
tempparams= { 'n_neighbors':knrange}
gstemp  =  GridSearchCV(knr, param_grid=tempparams, cv=8)

In [38]:
gstemp.fit(knnfeaturesscaled,age_imput_y)

GridSearchCV(cv=8, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_neighbors': range(1, 30)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [39]:
gstemp.best_score_

0.28690799720736249

In [40]:
gstemp.best_params_

{'n_neighbors': 12}

In [41]:
gstemp.cv_results_

{'mean_fit_time': array([ 0.0034264 ,  0.00333858,  0.00399062,  0.00402072,  0.00333983,
         0.00314447,  0.00324911,  0.00337353,  0.00413805,  0.00400874,
         0.00342283,  0.00213742,  0.00212824,  0.0021306 ,  0.0021244 ,
         0.00212306,  0.00212505,  0.00212067,  0.00212911,  0.00213766,
         0.00212109,  0.00211763,  0.00212318,  0.0021182 ,  0.00212729,
         0.00211379,  0.00211981,  0.00211805,  0.0021233 ]),
 'mean_score_time': array([ 0.10828689,  0.10810709,  0.1098308 ,  0.10976037,  0.10855818,
         0.1086376 ,  0.10763347,  0.1088011 ,  0.10998136,  0.10977477,
         0.10868087,  0.10536394,  0.10524747,  0.10524568,  0.10527015,
         0.10539642,  0.10529047,  0.10519704,  0.10528943,  0.10529518,
         0.10544813,  0.10527921,  0.10532475,  0.10534233,  0.10551393,
         0.10532543,  0.1053642 ,  0.10529026,  0.10535735]),
 'mean_test_score': array([-0.24159373,  0.11226062,  0.15913168,  0.21701374,  0.23349621,
         0.2514002